### Image Segmentation Project 1
###### The following program optimizes the ambrios functional using gradient descent method to use a gradieent descent method, the gradient of the functional needs

In [ ]:
@time begin
    
using Images, FileIO
using TestImages
using LinearAlgebra
using ImageView
using Base.MathConstants
    
end

In [ ]:
# set the image to segment.
uO = Gray.(testimage("lighthouse"));
uO = Gray.(load("faces1.jpg"))
m = size(uO)[1];
n = size(uO)[2];
u = rand(Gray,m,n);
v = rand(Gray,m,n);
#imshow(uO);

In [ ]:
# The following function is used for the
# gamma term which calculate the gradient
# of a certain pixel using finite differences.

function finite_diff_grad(M,x)
    
    partial_x = (M[x[1]+1,x[2]] - M[x[1]-1,x[2]]) / 2;
    partial_y = (M[x[1],x[2]+1] - M[x[1],x[2]-1]) / 2;
    
    return [partial_x, partial_y];
    
end;

In [ ]:
epsilon = 0.2;
gamma = 0.2;
alph = 0.2;
step = 0.5;

function gaussian_discrete(m,n,epsilon)
    grid = zeros(m,n)
    for i = 1:m
        for j = 1:n
            exponent = (i^2)/2+(j^2)/2;
            grid[i,j] = exp(-exponent);
        end;
    end;
    return grid;
end;

function g_epsilon(m,n,epsilon)
    phi = gaussian_discrete(m,n,epsilon);
    return phi;
end;

function partial_F_sim_u(u,uO,k)
    return 2*(u[k[1],k[2]]-uO[k[1],k[2]]);
end;

function partial_F_sim_v()
    return 0;
end;

function partial_F_Reg_At_v(u,v,k)
    
    partial = 2*v[k[1],k[2]]*norm(finite_diff_grad(u,k))^2;
    return partial;
end;

function partial_F_Reg_At_u(u,v,k)
    partial= 2*norm(finite_diff_grad(u,k))*v[k[1],k[2]]^2;
    return partial;
end;

function partial_F_At_u()
    return 0;
end;

function partial_F_At_v(v,k,epsilon)
    
    partial = 2*epsilon*norm(finite_diff_grad(v,k));
    partial += (1/(2*epsilon))*(v[k[1],k[2]]-1);
    return partial;
end;

function gradient_F_MSAT(u,v,alph,uO,gamma,k,epsilon)

    partial_u = partial_F_sim_u(u,uO,k);
    partial_u += alph*partial_F_Reg_At_u(u,v,k);
    partial_u += gamma*partial_F_At_u();
    
    partial_v = partial_F_sim_v();
    partial_v += alph*partial_F_Reg_At_v(u,v,k);
    partial_v += gamma*partial_F_At_v(v,k,epsilon);
    
    return [partial_u partial_v];
end;

function F_sim(u,uO,k)
    
     value = (u[k[1],k[2]]-uO[k[1],k[2]])^2
    return value;
end;

function F_Reg_At(u,v,k)
    
    value = norm(finite_diff_grad(u,k))^2
    value *= v[k[1],k[2]]^2;
    return value;
end;

function F_At(u,v,k,epsilon)
      
    value = epsilon*norm(finite_diff_grad(v,k));
    value += (1/(4*epsilon))*(v[k[1],k[2]]-1)^2
    return value;
end;

function F_MSAT(u,uO,v,k,gamma,alph,epsilon)
    return F_sim(u,uO,k)+F_Reg_At(u,v,k)+F_At(u,v,k,epsilon);
end;

#F_MSAT(u,uO,v,[10,17],gamma,alph,epsilon)

In [ ]:
@time begin
    
    threshold = 0.001;
    cur_error = threshold + 1;
    max_iterations = 100;
    curIter = 0;
    while(curIter < max_iterations && cur_error > threshold)
        for i = 3:m-2
            for j = 3:n-2
                k = [i,j];
                grad = gradient_F_MSAT(u,v,alph,uO,gamma,k,epsilon);
                u[i,j] = u[i,j] - step*grad[1];
                v[i,j] = v[i,j] - step*grad[2];
            end;
        end;
        cur_error = norm(u-uO);
        curIter += 1
    end;
end
print(cur_error);

In [ ]:
# The following blocks of code implements conjugate gradient descent
# the blocks of code have been taken from algorithms for optimzation textbook
# Fiboancci search, line search, and bracketing has been taken from algorithms for optimziation
# chapters 3 and 5.
imshow(v)

In [ ]:
# @time begin
    
#     max_iterations = 100
#     curIter = 0

#     while curIter < max_iterations
#         # Define the indices for which the gradient needs to be computed
#         indices = CartesianIndices((3:m-2, 3:n-2))

#         # Compute the gradients for all indices at once
#         grads = gradient_F_MSAT.(Ref(u), Ref(v), Ref(alph), Ref(uO), Ref(alph), Ref(gamma), indices)

#         # Reshape gradients and indices for broadcasting
#         grads_reshaped = hcat([grad[1] for grad in grads]...), hcat([grad[2] for grad in grads]...)
#         indices_reshaped = hcat([i for i in indices]...)

#         # Update u and v using array broadcasting
#         u[indices_reshaped] .-= step .* grads_reshaped[1]
#         v[indices_reshaped] .-= step .* grads_reshaped[2]

#         curIter += 1
#     end

# end

In [ ]:



function optimalStepSize(u,v,k,d,epsilon,gamma,alph)
    num = -2*d[1]*(u[k[1],k[2]]-uO[k[1],k[2]]);
    num -= 2*d[2]*alph*(norm(finite_diff_grad(u,k))^2)*v[k[1],k[2]];
    num -= d[2]*gamma/(2*epsilon)*(v[k[1],k[2]]-1);
    
    denom = 2*d[1]^2 + 2*alph*d[2]^2*(norm(finite_diff_grad(u,k))^2)*v[k[1],k[2]]
    denom += d[2]^2*gamma /(2*epsilon);
    α = num / denom;
    uv_ij = zeros(1,2);
    uv_ij[1,1] = u[k[1],k[2]] + α*d[1];
    uv_ij[1,2] = v[k[1],k[2]] + α*d[2];
    uv_ij = Gray.(uv_ij);
    #uv_ij = uv_ij + α.*d;
    return uv_ij;
end;

In [ ]:
abstract type DescentMethod end

mutable struct ConjugateGradientDescent <: DescentMethod
    d
    g
end
function init!(M::ConjugateGradientDescent, f, ∇f, k)
    M.g = ∇f(k)
    M.d = -M.g
    return M
end
function step!(M::ConjugateGradientDescent, u,v,∇f,k,epsilon,gamma,alph)
    d, g = M.d, M.g
    g′ = ∇f(k)
    β = max(0, dot(g′, g′-g)/(g⋅g))
    d′ = -g′ +  β*d;
    uv_t1 = optimalStepSize(u,v,k, d′,epsilon,gamma,alph);
    M.d, M.g = d′, g′
    return uv_t1
end;

In [ ]:
function conj_func(u,uO,v,gamma,alph,epsilon,obj)
    function f(uv_ij,k)
        u[k[1],k[2]] = uv_ij[1];
        v[k[1],k[2]] = uv_ij[2];
        return obj(u,uO,v,k,gamma,alph,epsilon);
    end;
    return f;
end;

function conj_grad_func(u,v,alph,uO,gamma,epsilon,func)
    function grad_f(k)
        return func(u,v,alph,uO,gamma,k,epsilon);
    end;
    return grad_f;
end;

In [ ]:
grad = conj_grad_func(u,v,alph,uO,gamma,epsilon,gradient_F_MSAT);
conjDesc = ConjugateGradientDescent([0,0],[0,0]);
#init!(conjDesc, nothing, grad, [3,3]);

In [ ]:
@time begin
    conjDesc = ConjugateGradientDescent(nothing,nothing);
    init!(conjDesc, nothing, grad, [3,3]);
    threshold = 0.001;
    cur_error = threshold + 1;
    max_iterations = 100;
    curIter = 0;
    while(curIter < max_iterations && cur_error > threshold)
        for i = 3:m-2
            for j = 3:n-2
                k = [i,j];
                updatedValue = step!(conjDesc,u,v,grad,k,epsilon,gamma,alph);
                u[i,j] = updatedValue[1];
                v[i,j] = updatedValue[2];
            end;
        end;
        cur_error = norm(u-uO);
        curIter += 1
        print(curIter,"\n");
    end;
end

In [ ]:
imshow(u);

In [ ]:
imshow(v);

In [ ]:
imshow(u)

In [ ]:
#change black to white and white black
# in the image, v is the contours
using Statistics
avg = mean(v)
v .= (v .< avg) .|> Int
imshow(v);

In [ ]:
#The following code has been taken from Morgan Romanelli homework 2
# in the cs268 optimization class with their permission

In [ ]:
mutable struct Adam <: DescentMethod
    α # learning rate
    γv # decay
    γs # decay
    ϵ # small value
    k # step counter
    v # 1st moment estimate
    s # 2nd moment estimate
end

Base.copy(s::Adam) = Adam(s.α,s.γv,s.γs,s.ϵ,s.k,s.v,s.s) 

function init!(M::Adam, f, ∇f, x, alph)
    M.k = 0
    M.v = zeros(length(x))
    M.s = zeros(length(x))
    return M
end

function step!(M::Adam, f, ∇f,pos,uv_ij)
    α, γv, γs, ϵ, k = M.α, M.γv, M.γs, M.ϵ, M.k
    s, v, g = M.s, M.v, ∇f(pos)
    v[:] = γv*v + (1-γv)*g
    s[:] = γs*s + (1-γs)*g.*g
    M.k = k += 1
    v_hat = v ./ (1 - γv^k)
    s_hat = s ./ (1 - γs^k)
    return uv_ij - α*v_hat ./ (sqrt.(s_hat) .+ ϵ)
end
grad = conj_grad_func(u,v,alph,uO,gamma,epsilon,gradient_F_MSAT);
alpha = 0.08
yv = 0.9
ys = 0.999
eps = 10^(-8)
a = Adam(alpha,yv,ys,eps,0,Gray.(zeros(1,2)),Gray.(zeros(1,2)))

In [ ]:
@time begin
    threshold = 0.001;
    cur_error = threshold + 1;
    max_iterations = 100;
    curIter = 0;
    while(curIter < max_iterations && cur_error > threshold)
        for i = 3:m-2
            for j = 3:n-2
                pos = [i,j];
                uv_ij = zeros(1,2);
                uv_ij[1,1] = u[i,j];
                uv_ij[1,2] = v[i,j];
                updatedValue = step!(a, nothing, grad,pos, Gray.(uv_ij));
                u[i,j] = updatedValue[1];
                v[i,j] = updatedValue[2];
            end;
        end;
        cur_error = norm(u-uO);
        curIter += 1
        print(curIter,"\n");
    end;
end

In [ ]:
print(cur_error)

In [ ]:
typeof(Gray.(zeros(1,2)))

In [ ]:
imshow(u)

In [ ]:
imshow(v)